In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os

# set env flags to catch BLAS used for scipy/numpy 
# to only use 1 cpu, n_cpus will be totally controlled by csky
if False:
    os.environ['MKL_NUM_THREADS'] = "1"
    os.environ['NUMEXPR_NUM_THREADS'] = "1"
    os.environ['OMP_NUM_THREADS'] = "1"
    os.environ['OPENBLAS_NUM_THREADS'] = "1"
    os.environ['VECLIB_MAXIMUM_THREADS'] = "1"

import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = 'w'
mpl.rcParams['savefig.facecolor'] = 'w'
import matplotlib.pyplot as plt
from matplotlib import colors, cm
import csky as cy
from csky import cext
import numpy as np
import astropy
import histlite as hl
import healpy
import healpy as hp
import socket
import pickle
from scipy import stats
import copy
healpy.disable_warnings()
plt.rc('figure', facecolor = 'w')
plt.rc('figure', dpi=100)

## Define Settings

In [ ]:
selection_version = 'version-001-p01'

host_name = socket.gethostname()

if 'cobalt' in host_name:
    print('Working on Cobalts')
    #data_prefix = '/data/user/ssclafani/data/cscd/final'
    ana_dir = '/data/user/ssclafani/data/analyses/'
    plot_dir = cy.utils.ensure_dir('/data/user/ssclafani/data/analyses/DNNCascadeCodeReview/unblinding_checks/plots/unblinding/galactic_plane_checks')
    
else:
    raise ValueError('Unknown host:', host_name)

In [ ]:
for dir_path in [plot_dir]:
    if not os.path.exists(dir_path):
        print('Creating directory:', dir_path)
        os.makedirs(dir_path)

## Load Data

In [ ]:
repo = cy.selections.Repository(local_root='/data/user/ssclafani/data/analyses')
specs = cy.selections.DNNCascadeDataSpecs.DNNC_10yr

In [ ]:
%%time

ana = cy.get_analysis(
    repo, selection_version, specs, 
    #gammas=np.r_[0.1:6.01:0.125],
)

## Setup Analysis

In [ ]:
import sys
sys.path.insert(0, '../..')

import config as cg

cg.base_dir = '/data/user/mhuennefeld/data/analyses/unblinding_v1.0.1/'

In [ ]:
import pandas as pd

In [ ]:
source_list = pd.read_pickle('/home/ssclafani/DNNCascade/catalogs/Source_List_DNNC.pickle')

In [ ]:
src_list_file = os.path.join(cg.catalog_dir, 'Source_List_DNNC.npy')
sourcelist = np.load(src_list_file, allow_pickle=True)

# load correlated MultiTrialRunner background trials
print('Loading correlated background trials...')
base_dir = os.path.join('/data/user/mhuennefeld/data/analyses/unblinding_v1.0.1/ps/correlated_trials/')
bgfile_corr = '{}/correlated_bg.npy'.format(base_dir)
trials_corr = np.load(bgfile_corr, allow_pickle=True)
bg_corr = cy.dists.TSD(trials_corr)

# load bkg trials at source declinations
print('Loading background trials at source declinations...')
bgfile = '{}/pretrial_bgs.npy'.format(base_dir)
bgs = np.load(bgfile, allow_pickle=True)                             


In [ ]:
base_dir = '/data/user/mhuennefeld/data/analyses/unblinding_v1.0.1/'
results = np.load(base_dir + '/ps/results/fulllist_unblinded.npy', allow_pickle=True)

In [ ]:
sig = np.load('/data/user/ssclafani/data/analyses/baseline_unblind_v1.0.0/ps/correlated_trials/sourcelist_sig.npy' , allow_pickle=True)

In [ ]:

def get_tr(dec, ra,  gamma):
    src = cy.utils.sources(ra, dec, deg=False)                  
    conf = cg.get_ps_conf(src=src, gamma=gamma)
    tr = cy.get_trial_runner(ana=ana, conf=conf, TRUTH=False)
    return tr, src
def get_ul(gamma, verbose=False):
    uls90 = []
    for i, source in enumerate(sourcelist):
        tr, src = get_tr(
        dec=np.radians(source[2]),
        ra=np.radians(source[1]),
        gamma=gamma)
        ts = results[0][i][0]
        bg = bgs[i]
        trials = sig[i]['gamma'][gamma]['cutoff_TeV'][np.inf]['nsig']

        trials[0] = bg.trials
        print('Source {}: Getting fraction of trials > {:.4}: For Gamma = {}'.format(i, ts, gamma))
        ul90 = tr.find_n_sig(ts, 0.9, trials=trials, logging=verbose, tol=.1)
        ul90['flux'] = tr.to_E2dNdE(ul90['n_sig'], E0=100, unit=1e3)
        uls90.append(ul90['flux'])
        print(ul90['flux'], ul90['n_sig'])
    return uls90

In [ ]:
uls90_E2 = get_ul(2.0, verbose=False)

In [ ]:
np.save('sourcelist_UL90_E2_patch_01.npy' ,uls90_E2)

In [ ]:
uls90_E3 = get_ul(3.0, verbose=False)

In [ ]:
np.save('sourcelist_UL90_E3_patch_01.npy' ,uls90_E3)